In [1]:
import os
os.chdir('../')

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [4]:
from ml_project.constants import *
from ml_project.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
            self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )
        return data_transformation_config


In [5]:
import os
import pandas as pd
from ml_project import logger
from ml_project.utils.common import get_size
from sklearn.model_selection import train_test_split

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_split(self):
        data = pd.read_csv(self.config.data_path)
        train, test = train_test_split(data, random_state=42)
        train.to_csv(os.path.join(self.config.root_dir, 'train.csv'), index=False)
        test.to_csv(os.path.join(self.config.root_dir, 'test.csv'), index=False)

        logger.info('Splitted data into training and test sets')
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)

[2024-01-14 19:18:38,313: INFO: utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2024-01-14 19:18:38,314: INFO: utils: NumExpr defaulting to 8 threads.]


In [6]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_split()
except Exception as e:
    raise e

[2024-01-14 19:19:41,471: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-14 19:19:41,473: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-14 19:19:41,478: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-14 19:19:41,480: INFO: common: created directory at: artifacts]
[2024-01-14 19:19:41,481: INFO: common: created directory at: artifacts/data_transformation]
[2024-01-14 19:19:41,517: INFO: 767086754: Splitted data into training and test sets]
[2024-01-14 19:19:41,518: INFO: 767086754: (1199, 12)]
[2024-01-14 19:19:41,519: INFO: 767086754: (400, 12)]
(1199, 12)
(400, 12)
